<a href="https://colab.research.google.com/github/uplearni/CreditLimitReccomender/blob/main/CreditLimitProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#libraries

#for data manipulation
import pandas as pd;
import numpy as np;
import io
import warnings
warnings.filterwarnings('ignore')

#for data visualisation
import matplotlib.pyplot as plt;
import seaborn as sns;
from matplotlib.cbook import boxplot_stats

#for data modelling
from sklearn.model_selection import train_test_split
#models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
#for model evaluation
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
#for model preprocessing
from sklearn.preprocessing import PowerTransformer
#for statistical analysis
import statsmodels.api as sm

In [ ]:
#dataset
url = ''
Credit_Data= pd.read_csv(url)